# Introduction

## Z3 and Z3py resources

For this problem you will be solving constraint satisfaction and logic problems using the Z3 SMT solver via the z3py python library.

The demo code using z3py we went over in class is here: [link](https://drive.google.com/drive/folders/16HQXiwdcaman1IpC7H6vSJ71QQQ-aL2z?usp=sharing)

Online help for z3py is here: [link](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## Installation

We first install to get the z3solver library using pip and test that works.

In [13]:
!pip install z3-solver

In [14]:
# Run the first example from the z3py guide as a test: https://ericpony.github.io/z3py-tutorial/guide-examples.htm
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


# Sudoku as a constraint satisfaction problem (CSP)

Sudoku is a popular number-placement puzzle that originated in France in the end of the 19th century.  Modern Sudoku was likely invented by Howard Garns from Connersville, Indiana and was first published in 1979 under the name *Number Place*. The objective of the puzzle is to place numbers 1-9 on a 9$\times$9 grid, such that each number occurs only once in every row, every column, and every of the nine 3$\times$3 sub-grids that compose the main grid.
Sudoku puzzles are grids that have been partially occupied with numbers. The task is then to occupy the remaining fields in such a way that the constraints on rows, columns, and sub-grids are satisfied.
For more information about Sudoku refer to its Wikipedia page at http://en.wikipedia.org/wiki/Sudoku.

This problem has two parts. In the first part, you will write the boolean constraints in mathematical notation for solving a Sudoku puzzle. In the second part, you will write code and invoke the Z3 solver to solve the Sudoku instance and answer various questions about the solution.

## Part 1 (Constraints)
In text, define and write constraints over **boolean** variables corresponding to each number being in each cell. For example, you may use $X_{i,j}^k$ as the variable that is true if and only if the number $k$ is in row $i$ and column $j$ (the variable is true if $k$ is in cell $(i, j)$ and false otherwise). Now write the following boolean constaints over these variables:

* Write the boolean formula for the constraints that the number 5 can occur at most once in the first row. **(1 point)**
* Write the boolean formula for the constraints that the number 6 can occur at most once in the first column. **(1 point)**
* Write the boolean formula for the constraints that the number 9 can occur at most once in the top left 3$\times$3 sub-grid. **(1 point)**

Constraints:

* The number $k$ in row $i$ and column $j$ that is part of the initial grid state $I$ cannot be changed. This can be expressed as $\forall_{(i,j,k) \in I} (X_{i,j}^k)$.
* For all rows $i$ and all columns $j$, row $i$ and column $j$ must contain at least one number $k$. This can be expressed as $\forall_{i,j} (\exists_k (X_{i,j}^k))$.
* For all rows $i$, all columns $j$, and all numbers $k$, row $i$ and column $j$ cannot contain number $k$ and number $k'$ as well, where $k' \neq k$. This can be expressed as $\forall_{i,j,k} (X_{i,j}^k \Rightarrow \forall_{k' \neq k} (\neg X_{i,j}^{k'}))$.
* For all rows $i$, all columns $j$, and all numbers $k$, the number $k$ in row $i$ and column $j$ cannot be in row $i$ and column $j'$ as well, where $j' \neq j$. This can be expressed as $\forall_{i,j,k} (X_{i,j}^k \Rightarrow \forall_{j' \neq j} (\neg X_{i,j'}^k))$.
* For all rows $i$, all columns $j$, and all numbers $k$, the number $k$ in row $i$ and column $j$ cannot be in row $i'$ and column $j$ as well, where $i' \neq i$. This can be expressed as $\forall_{i,j,k} (X_{i,j}^k \Rightarrow \forall_{i' \neq i} (\neg X_{i',j}^k))$.
* For all positions $(i,j)$ of the form $(row,column)$ contained within each sub-grid $G$ and all numbers $k$, the number $k$ in row $i$ and column $j$ cannot be in row $i'$ and column $j'$ as well, where $(i', j') \neq (i, j)$. This can be expressed as $\forall_{G} (\forall_{(i,j) \in G,k} (X_{i,j}^k \Rightarrow \forall_{(i',j') \neq (i,j)} (\neg X_{i',j'}^k)))$.

Boolean Formulas (with (1,1) being the top-left corner of the grid):

* $\forall_{j} (X_{1,j}^5 \Rightarrow \forall_{j' \neq j} (\neg X_{1,j'}^5))$
* $\forall_{i} (X_{i,1}^6 \Rightarrow \forall_{i' \neq i} (\neg X_{i',1}^6))$
* $\forall_{(i,j) \in G} (X_{i,j}^9 \Rightarrow \forall_{(i',j') \neq (i,j)} (\neg X_{i',j'}^9))$, where $G = \{(1,1), (1,2), (1,3), (2,1), (2,2), (2,3), (3,1), (3,2), (3,3)\}$

## Part 2 (Coding)

Encode the above constraints and all the other ones as a CSP using z3py and solve the Sudoku instance given in the figure below. **Use only boolean variables** and do not use the **Distinct** function.

* Provide the code to solve that solves the given problem instance using z3py and only boolean variables. The code should output a reasonable visualization of the solution, for example printed in text. **(1 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/sudoku1.jpg" width="400"/>



In [15]:
# z3py code to solve above Sudoku goes here. Do not use Int(*), only Bool(*)
from z3 import *

def solve_sudoku(grid, show=0):
  """Solves a sudoku grid using the last solution found and prints the number of solutions found."""

  # create a Solver
  s = Solver()

  # for all rows i
  for i in range(len(grid)):
    # for all columns j
    for j in range(len(grid[0])):

      # if grid position is not occupied
      if grid[i][j] == 0:

        # for all numbers k
        for k in range(1, 10):
          # add row constraint
          s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{x,j,k}")) for x in range(9) if x != i])))
          # add column constraint
          s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{i,x,k}")) for x in range(9) if x != j])))
          # add at most one number per grid position constraint
          s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{i,j,x}")) for x in range(1, 10) if x != k])))

        # add at least one number per grid position constraint
        s.add(Or(*[Bool(f"X_{i,j,k}") for k in range(1, 10)]))

      # if grid position is occupied
      else:

        # add initial grid state constraint
        s.add(Bool(f"X_{i,j,grid[i][j]}"))

  # iterate over sub-grid row number upper limits
  for a in range(3, 10, 3):
    # iterate over sub-grid column number upper limits
    for b in range(3, 10, 3):
      # for all sub-grid rows
      for i in range(a - 3, a):
        # for all sub-grid columns
        for j in range(b - 3, b):
          # for all numbers k
          for k in range(1, 10):
            # add sub-grid constraint
            s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{x,y,k}")) for x in range(a - 3, a) for y in range(b - 3, b) if x != i or y != j])))

  # number of solutions
  num = 0

  # loop until no other solution is found
  while s.check() == sat:

    # increment number of solutions
    num += 1

    # get model
    m = s.model()

    # constraint to find a different solution
    constraint = None

    # iterate over variables in the model
    for v in m:

      # if the variable is true
      if m[v]:

        # print the variable
        # print(f"{v}: {m[v]}")

        # get name of variable
        name = v.name()

        # extract i, j, and k from name of variable
        ijk = (int(name[3]), int(name[6]), int(name[9]))

        # add number to grid
        grid[ijk[0]][ijk[1]] = ijk[2]

        # build constraint to find a different solution
        if constraint is None:
          constraint = Bool(name)
        else:
          constraint = And(constraint, Bool(name))

    # add different solution constraint
    s.add(Not(constraint))

    # show up to show solutions
    if num <= show:
      show_sudoku(grid)

  # print number of solutions found
  print("Solutions found:", num)

def show_sudoku(grid):
  """Prints a sudoku grid."""

  print("-------------------------------")
  for i in range(len(grid)):
    print("|", end="")
    for j in range(len(grid[0])):
      print("", grid[i][j], "", end="")
      if (j + 1) % 3 == 0:
        print("|", end="")
    print()
    if (i + 1) % 3 == 0:
      print("-------------------------------")

grid = [[0, 1, 0, 4, 0, 2, 0, 5, 0],
        [5, 0, 0, 0, 0, 0, 0, 0, 6],
        [0, 0, 0, 3, 0, 1, 0, 0, 0],
        [7, 0, 5, 0, 0, 0, 4, 0, 8],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 8, 0, 0, 0, 5, 0, 9],
        [0, 0, 0, 9, 0, 6, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 0, 0, 2],
        [0, 7, 0, 1, 0, 3, 0, 4, 0]]

solve_sudoku(grid, show=3)

-------------------------------
| 3  1  9 | 4  6  2 | 8  5  7 |
| 5  2  4 | 7  9  8 | 1  3  6 |
| 8  6  7 | 3  5  1 | 9  2  4 |
-------------------------------
| 7  3  5 | 2  1  9 | 4  6  8 |
| 1  9  6 | 8  4  5 | 2  7  3 |
| 2  4  8 | 6  3  7 | 5  1  9 |
-------------------------------
| 4  5  3 | 9  2  6 | 7  8  1 |
| 6  8  1 | 5  7  4 | 3  9  2 |
| 9  7  2 | 1  8  3 | 6  4  5 |
-------------------------------
Solutions found: 1



### Uniqueness
Is your solution unique? Prove it with a Z3py solver or provide a second solution. **(1 point)**

In [16]:
# The solution is unique as shown in the printed output below.

grid = [[0, 1, 0, 4, 0, 2, 0, 5, 0],
        [5, 0, 0, 0, 0, 0, 0, 0, 6],
        [0, 0, 0, 3, 0, 1, 0, 0, 0],
        [7, 0, 5, 0, 0, 0, 4, 0, 8],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 8, 0, 0, 0, 5, 0, 9],
        [0, 0, 0, 9, 0, 6, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 0, 0, 2],
        [0, 7, 0, 1, 0, 3, 0, 4, 0]]

solve_sudoku(grid, show=3)

-------------------------------
| 3  1  9 | 4  6  2 | 8  5  7 |
| 5  2  4 | 7  9  8 | 1  3  6 |
| 8  6  7 | 3  5  1 | 9  2  4 |
-------------------------------
| 7  3  5 | 2  1  9 | 4  6  8 |
| 1  9  6 | 8  4  5 | 2  7  3 |
| 2  4  8 | 6  3  7 | 5  1  9 |
-------------------------------
| 4  5  3 | 9  2  6 | 7  8  1 |
| 6  8  1 | 5  7  4 | 3  9  2 |
| 9  7  2 | 1  8  3 | 6  4  5 |
-------------------------------
Solutions found: 1


### More Uniqueness
If you delete the 1 in the top left box of the Sudoku problem above, how many unique solutions are there? Hint: should be less than 5000. **(1 point)**

In [17]:
grid = [[0, 0, 0, 4, 0, 2, 0, 5, 0],
        [5, 0, 0, 0, 0, 0, 0, 0, 6],
        [0, 0, 0, 3, 0, 1, 0, 0, 0],
        [7, 0, 5, 0, 0, 0, 4, 0, 8],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 8, 0, 0, 0, 5, 0, 9],
        [0, 0, 0, 9, 0, 6, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 0, 0, 2],
        [0, 7, 0, 1, 0, 3, 0, 4, 0]]

solve_sudoku(grid, show=3)

-------------------------------
| 3  9  7 | 4  6  2 | 8  5  1 |
| 5  1  4 | 7  9  8 | 3  2  6 |
| 8  2  6 | 3  5  1 | 7  9  4 |
-------------------------------
| 7  3  5 | 2  1  9 | 4  6  8 |
| 1  6  9 | 8  4  5 | 2  7  3 |
| 2  4  8 | 6  3  7 | 5  1  9 |
-------------------------------
| 4  5  3 | 9  2  6 | 1  8  7 |
| 6  8  1 | 5  7  4 | 9  3  2 |
| 9  7  2 | 1  8  3 | 6  4  5 |
-------------------------------
-------------------------------
| 1  8  7 | 4  6  2 | 9  5  3 |
| 5  3  4 | 8  9  7 | 1  2  6 |
| 9  2  6 | 3  5  1 | 7  8  4 |
-------------------------------
| 7  6  5 | 2  3  9 | 4  1  8 |
| 4  9  3 | 5  1  8 | 2  6  7 |
| 2  1  8 | 6  7  4 | 5  3  9 |
-------------------------------
| 3  5  2 | 9  4  6 | 8  7  1 |
| 6  4  1 | 7  8  5 | 3  9  2 |
| 8  7  9 | 1  2  3 | 6  4  5 |
-------------------------------
-------------------------------
| 1  8  7 | 4  6  2 | 9  5  3 |
| 5  4  3 | 8  9  7 | 1  2  6 |
| 9  2  6 | 3  5  1 | 7  8  4 |
-------------------------------
| 7  6  

# Miracle Sudoku

Using z3py and only boolean variables, solve the Miracle Sudoku ([spoiler](https://www.youtube.com/watch?v=yKf9aUIxdb4)).
In this puzzle normal Sudoku rules apply in addition to the following rules:
* Any two cells separated by a knight's move from chess (moving forward two boxes and over one, in any direction) cannot contain the same digit (see image below).
* Any two cells separated by a king's move from chess (one box in any direction including diagonals) cannot contain the same digit.
* Any two orthogonally adjacent (up, down, left, or right) cells cannot contain consecutive digits.

### Knight's move constraint image
<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/knights.jpg" width="400"/>

The knight's move constraint would mean the yellow boxes cannot contain a 1.


## Miracle Sudoku Starting Information
Provide the code to solve the puzzle given below. The code should output a reasonable visualization of the solution, for example using text. **(2 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/miracle.jpg" width="400"/>

In [18]:
# Solution code for Miracle Sudoku using z3py
from z3 import *

def solve_miracle_sudoku(grid, show=0):
  """Solves a miracle sudoku grid using the last solution found and prints the number of solutions found."""

  # create a Solver
  s = Solver()

  # for all rows i
  for i in range(len(grid)):
    # for all columns j
    for j in range(len(grid[0])):

      # if grid position is not occupied
      if grid[i][j] == 0:

        # for all numbers k
        for k in range(1, 10):
          # add row constraint
          s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{x,j,k}")) for x in range(9) if x != i])))
          # add column constraint
          s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{i,x,k}")) for x in range(9) if x != j])))
          # add at most one number per grid position constraint
          s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{i,j,x}")) for x in range(1, 10) if x != k])))
          # add knight's move constraint
          s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{i+x,j+y,k}")) for x in [-2, -1, 1, 2] for y in [-2, -1, 1, 2] if abs(x) != abs(y) and 0 <= i + x <= 8 and 0 <= j + y <= 8])))
          # add king's move constraint
          s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{i+x,j+y,k}")) for x in [-1, 0, 1] for y in [-1, 0, 1] if x != 0 and y != 0 and 0 <= i + x <= 8 and 0 <= j + y <= 8])))
          # add orthogonal constraint
          s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{i+x,j+y,k+z}")) for x in [-1, 0, 1] for y in [-1, 0, 1] for z in [-1, 1] if abs(x) != abs(y) and 0 <= i + x <= 8 and 0 <= j + y <= 8 and 1 <= k + z <= 9])))

        # add at least one number per grid position constraint
        s.add(Or(*[Bool(f"X_{i,j,k}") for k in range(1, 10)]))

      # if grid position is occupied
      else:

        # add initial grid state constraint
        s.add(Bool(f"X_{i,j,grid[i][j]}"))

  # iterate over sub-grid row number upper limits
  for a in range(3, 10, 3):
    # iterate over sub-grid column number upper limits
    for b in range(3, 10, 3):
      # for all sub-grid rows
      for i in range(a - 3, a):
        # for all sub-grid columns
        for j in range(b - 3, b):
          # for all numbers k
          for k in range(1, 10):
            # add sub-grid constraint
            s.add(Implies(Bool(f"X_{i,j,k}"), And(*[Not(Bool(f"X_{x,y,k}")) for x in range(a - 3, a) for y in range(b - 3, b) if x != i or y != j])))

  # number of solutions
  num = 0

  # loop until no other solution is found
  while s.check() == sat:

    # increment number of solutions
    num += 1

    # get model
    m = s.model()

    # constraint to find a different solution
    constraint = None

    # iterate over variables in the model
    for v in m:

      # if the variable is true
      if m[v]:

        # print the variable
        # print(f"{v}: {m[v]}")

        # get name of variable
        name = v.name()

        # extract i, j, and k from name of variable
        ijk = (int(name[3]), int(name[6]), int(name[9]))

        # add number to grid
        grid[ijk[0]][ijk[1]] = ijk[2]

        # build constraint to find a different solution
        if constraint is None:
          constraint = Bool(name)
        else:
          constraint = And(constraint, Bool(name))

    # add different solution constraint
    s.add(Not(constraint))

    # show up to show solutions
    if num <= show:
      show_sudoku(grid)

  # print number of solutions found
  print("Solutions found:", num)

grid = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]

solve_miracle_sudoku(grid, show=3)

-------------------------------
| 4  8  3 | 7  2  6 | 1  5  9 |
| 7  2  6 | 1  5  9 | 4  8  3 |
| 1  5  9 | 4  8  3 | 7  2  6 |
-------------------------------
| 8  3  7 | 2  6  1 | 5  9  4 |
| 2  6  1 | 5  9  4 | 8  3  7 |
| 5  9  4 | 8  3  7 | 2  6  1 |
-------------------------------
| 3  7  2 | 6  1  5 | 9  4  8 |
| 6  1  5 | 9  4  8 | 3  7  2 |
| 9  4  8 | 3  7  2 | 6  1  5 |
-------------------------------
Solutions found: 1




---


# Propositional Logic

Next, we will use truth tables and z3py to prove logical entailment.

## Truth Tables

Using markdown tables and bold to show relevant rows, use the truth table method to show logical truths.


### Example

Here's an example that shows $p, p \rightarrow q \models q$.  

| p | q | p -> q |
|-------|-------|---------------------|
| **T**     | **T**     | **T**                   |
| T     | F     | F                   |
| F     | T     | T                   |
| F     | F     | T                   |

Since $q$ is true whenever $p$ and $p \rightarrow q$ are true, the expression is true.

### Questions

Prove that the following semantic entailment expressions are true by using the truth table approach.  Bold the appropriate rows **(1 point)**

 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$
 * $\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$
 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$

In [19]:
# create list of truth values
truth_values = [True, False]

# create function to extract first letter from boolean
letter = lambda x: str(x)[0]

# define table headers
expr = ["p", "q", "r", "s", "t", "¬p", "¬s", "p → q", "(p → q) → r", "s → ¬p", "t → q"]

# print table header
print("", *expr, "", sep="|")
print("|-"*11, "|", sep="")

# iterate through all combinations of truth values for p, q, r, s, t
for p in truth_values:
  for q in truth_values:
    for r in truth_values:
      for s in truth_values:
        for t in truth_values:

          # find truth values of each row of the truth table
          row = [p, q, r, s, t, not p, not s, not p or q, not(not p or q) or r, not s or not p, not t or q]

          # create dictionary to map between expressions to truth values
          table = dict(zip(expr, row))

          # check if the left side of the entailment expression is true
          bold = table["(p → q) → r"] and table["s → ¬p"] and table["t"] and table["¬s"] and table["t → q"]

          # print the row in bold or not depending on truth value of left side of entailment expression
          if bold:
            print("|**", end="")
            print(*map(letter, row), sep="**|**", end="")
            print("**|")
          else:
            print("", *map(letter, row), "", sep="|")

# print divider between the tables
print("\n---\n")

# define table headers
expr = ["p", "q", "q → p", "p → (q → p)", "p → (p → (q → p))", "q → (p → (p → (q → p)))"]

# print table header
print("", *expr, "", sep="|")
print("|-"*6, "|", sep="")

# iterate through all combinations of truth values for p, q
for p in truth_values:
  for q in truth_values:

    # find truth values of each row of the truth table
    qp = not q or p
    pqp = not p or qp
    ppqp = not p or pqp
    qppqp = not q or ppqp
    row = [p, q, qp, pqp, ppqp, qppqp]

    # empty set is true
    bold = True

    # print the row in bold or not depending on truth value of left side of entailment expression
    if bold:
      print("|**", end="")
      print(*map(letter, row), sep="**|**", end="")
      print("**|")
    else:
      print("", *map(letter, row), "", sep="|")

|p|q|r|s|t|¬p|¬s|p → q|(p → q) → r|s → ¬p|t → q|
|-|-|-|-|-|-|-|-|-|-|-|
|T|T|T|T|T|F|F|T|T|F|T|
|T|T|T|T|F|F|F|T|T|F|T|
|**T**|**T**|**T**|**F**|**T**|**F**|**T**|**T**|**T**|**T**|**T**|
|T|T|T|F|F|F|T|T|T|T|T|
|T|T|F|T|T|F|F|T|F|F|T|
|T|T|F|T|F|F|F|T|F|F|T|
|T|T|F|F|T|F|T|T|F|T|T|
|T|T|F|F|F|F|T|T|F|T|T|
|T|F|T|T|T|F|F|F|T|F|F|
|T|F|T|T|F|F|F|F|T|F|T|
|T|F|T|F|T|F|T|F|T|T|F|
|T|F|T|F|F|F|T|F|T|T|T|
|T|F|F|T|T|F|F|F|T|F|F|
|T|F|F|T|F|F|F|F|T|F|T|
|T|F|F|F|T|F|T|F|T|T|F|
|T|F|F|F|F|F|T|F|T|T|T|
|F|T|T|T|T|T|F|T|T|T|T|
|F|T|T|T|F|T|F|T|T|T|T|
|**F**|**T**|**T**|**F**|**T**|**T**|**T**|**T**|**T**|**T**|**T**|
|F|T|T|F|F|T|T|T|T|T|T|
|F|T|F|T|T|T|F|T|F|T|T|
|F|T|F|T|F|T|F|T|F|T|T|
|F|T|F|F|T|T|T|T|F|T|T|
|F|T|F|F|F|T|T|T|F|T|T|
|F|F|T|T|T|T|F|T|T|T|F|
|F|F|T|T|F|T|F|T|T|T|T|
|F|F|T|F|T|T|T|T|T|T|F|
|F|F|T|F|F|T|T|T|T|T|T|
|F|F|F|T|T|T|F|T|F|T|F|
|F|F|F|T|F|T|F|T|F|T|T|
|F|F|F|F|T|T|T|T|F|T|F|
|F|F|F|F|F|T|T|T|F|T|T|

---

|p|q|q → p|p → (q → p)|p → (p → (q → p))|q → (p → (p → (q → p)))|

|p|q|r|s|t|¬p|¬s|p → q|(p → q) → r|s → ¬p|t → q|
|-|-|-|-|-|-|-|-|-|-|-|
|T|T|T|T|T|F|F|T|T|F|T|
|T|T|T|T|F|F|F|T|T|F|T|
|**T**|**T**|**T**|**F**|**T**|**F**|**T**|**T**|**T**|**T**|**T**|
|T|T|T|F|F|F|T|T|T|T|T|
|T|T|F|T|T|F|F|T|F|F|T|
|T|T|F|T|F|F|F|T|F|F|T|
|T|T|F|F|T|F|T|T|F|T|T|
|T|T|F|F|F|F|T|T|F|T|T|
|T|F|T|T|T|F|F|F|T|F|F|
|T|F|T|T|F|F|F|F|T|F|T|
|T|F|T|F|T|F|T|F|T|T|F|
|T|F|T|F|F|F|T|F|T|T|T|
|T|F|F|T|T|F|F|F|T|F|F|
|T|F|F|T|F|F|F|F|T|F|T|
|T|F|F|F|T|F|T|F|T|T|F|
|T|F|F|F|F|F|T|F|T|T|T|
|F|T|T|T|T|T|F|T|T|T|T|
|F|T|T|T|F|T|F|T|T|T|T|
|**F**|**T**|**T**|**F**|**T**|**T**|**T**|**T**|**T**|**T**|**T**|
|F|T|T|F|F|T|T|T|T|T|T|
|F|T|F|T|T|T|F|T|F|T|T|
|F|T|F|T|F|T|F|T|F|T|T|
|F|T|F|F|T|T|T|T|F|T|T|
|F|T|F|F|F|T|T|T|F|T|T|
|F|F|T|T|T|T|F|T|T|T|F|
|F|F|T|T|F|T|F|T|T|T|T|
|F|F|T|F|T|T|T|T|T|T|F|
|F|F|T|F|F|T|T|T|T|T|T|
|F|F|F|T|T|T|F|T|F|T|F|
|F|F|F|T|F|T|F|T|F|T|T|
|F|F|F|F|T|T|T|T|F|T|F|
|F|F|F|F|F|T|T|T|F|T|T|

*As shown by the bolded rows, r is true whenever (p→q)→r,s→¬p,t,¬s,t→q are all true, so the entailment expression (p→q)→r,s→¬p,t,¬s,t→q ⊨ r is true.

---

|p|q|q → p|p → (q → p)|p → (p → (q → p))|q → (p → (p → (q → p)))|
|-|-|-|-|-|-|
|**T**|**T**|**T**|**T**|**T**|**T**|
|**T**|**F**|**T**|**T**|**T**|**T**|
|**F**|**T**|**F**|**T**|**T**|**T**|
|**F**|**F**|**T**|**T**|**T**|**T**|

*As shown by the bolded rows, q→(p→(p→(q→p))) is true in all possible worlds, so the entailment expression ∅ ⊨ q→(p→(p→(q→p))) is true.



---


 ## Logical Entailment using z3py

 Use Z3py to prove the above three entailments are true using satisfiability. See section 7.5 in the book for details. **(1 point)**

In [20]:
# create a Solver
solver = Solver()

# create booleans
p, q, r, s, t = Bools("p q r s t")

# left side of entailment (alpha)
alpha = And(Implies(Implies(p, q), r), Implies(s, Not(p)), t, Not(s), Implies(t, q))

# right side of entailment (beta)
beta = r

# add α ∧ ¬β constraint
solver.add(And(alpha, Not(beta)))

# check for unsat
print("For (p→q)→r,s→¬p,t,¬s,t→q ⊨ r:")
print("(p→q)→r,s→¬p,t,¬s,t→q ∧ ¬r is", solver.check(), "\n")


# create a Solver
solver = Solver()

# create booleans
p, q = Bools("p q")

# left side of entailment (alpha)
alpha = False

# right side of entailment (beta)
beta = Implies(q, Implies(p, Implies(p, Implies(q, p))))

# add α ∧ ¬β constraint
solver.add(And(alpha, Not(beta)))

# check for unsat
print("For ∅ ⊨ q→(p→(p→(q→p))):")
print("∅ ∧ ¬(q→(p→(p→(q→p)))) is", solver.check(), "\n")


print("α ⊨ β iff α ∧ ¬β is unsatisfiable.")
print("α ∧ ¬β is unsatisfiable for the above entailment expressions of the form α ⊨ β.")
print("Therefore, the entailment expressions are true.")

For (p→q)→r,s→¬p,t,¬s,t→q ⊨ r:
(p→q)→r,s→¬p,t,¬s,t→q ∧ ¬r is unsat 

For ∅ ⊨ q→(p→(p→(q→p))):
∅ ∧ ¬(q→(p→(p→(q→p)))) is unsat 

α ⊨ β iff α ∧ ¬β is unsatisfiable.
α ∧ ¬β is unsatisfiable for the above entailment expressions of the form α ⊨ β.
Therefore, the entailment expressions are true.
